# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://make.technology")
ed.links

['#main',
 'javascript: void(0);',
 'https://make.technology/',
 'https://make.technology/contact-us/',
 'https://make.technology/about/',
 'https://make.technology/webinars/',
 'https://make.technology/projects/',
 'https://make.technology/articles/',
 'https://make.technology/careers/',
 'javascript: void(0);',
 'https://make.technology/contact-us/',
 'javascript: void(0);',
 'javascript: void(0);',
 "javascript: document.getElementById('searchform').submit();",
 '#main',
 'https://make.technology/projects/concepcion-industrial-corporation/',
 'https://make.technology/projects/concepcion-industrial-corporation/',
 'https://make.technology/projects/concepcion-industrial-corporation/',
 'https://make.technology/projects/generali/',
 'https://make.technology/projects/generali/',
 'https://make.technology/projects/generali/',
 'https://make.technology/projects/king-jewelers/',
 'https://make.technology/projects/king-jewelers/',
 'https://make.technology/projects/king-jewelers/',
 'https:

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://make.technology - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main
javascript: void(0);
https://make.technology/
https://make.technology/contact-us/
https://make.technology/about/
https://make.technology/webinars/
https://make.technology/projects/
https://make.technology/articles/
https://make.technology/careers/
javascript: void(0);
https://make.technology/contact-us/
javascript: void(0);
javascript: void(0);
javascript: document.getElementById('searchform').submit();
#main
https://make.technology/projects/concepcion-industrial-corporation/
https://make.technology/projects/concepcion-industrial-corporation/
https://make.technology/projects/concepcion-industrial-corporation/
https://make.technology/projects/generali/
https://make.technology/projects/gene

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://make.technology")
huggingface.links

['#main',
 'javascript: void(0);',
 'https://make.technology/',
 'https://make.technology/contact-us/',
 'https://make.technology/about/',
 'https://make.technology/webinars/',
 'https://make.technology/projects/',
 'https://make.technology/articles/',
 'https://make.technology/careers/',
 'javascript: void(0);',
 'https://make.technology/contact-us/',
 'javascript: void(0);',
 'javascript: void(0);',
 "javascript: document.getElementById('searchform').submit();",
 '#main',
 'https://make.technology/projects/concepcion-industrial-corporation/',
 'https://make.technology/projects/concepcion-industrial-corporation/',
 'https://make.technology/projects/concepcion-industrial-corporation/',
 'https://make.technology/projects/generali/',
 'https://make.technology/projects/generali/',
 'https://make.technology/projects/generali/',
 'https://make.technology/projects/king-jewelers/',
 'https://make.technology/projects/king-jewelers/',
 'https://make.technology/projects/king-jewelers/',
 'https:

In [11]:
get_links("https://make.technology")

{'links': [{'type': 'about page', 'url': 'https://make.technology/about/'},
  {'type': 'careers page', 'url': 'https://make.technology/careers/'},
  {'type': 'contact page', 'url': 'https://make.technology/contact-us/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://make.technology"))

Found links: {'links': [{'type': 'about page', 'url': 'https://make.technology/about/'}, {'type': 'careers page', 'url': 'https://make.technology/careers/'}, {'type': 'contact page', 'url': 'https://make.technology/contact-us/'}]}
Landing page:
Webpage Title:
Make Technology - Make Technology
Webpage Contents:
Skip to content
Contact Us
About
Webinars
Projects
Articles
Careers
Search
Ready to make something exciting?
Let's go
Cancel
Search
Made to
last
We craft digital experiences that inspire, inform and entertain people.
Discover what we do
UI Design
Concepcion Industrial Corporation
A website redesign for one of the leading air conditioning and refrigeration companies in the Philippines
View project
Workshop
Generali
Generali, a Fortune 100 company, is a major player in the global insurance industry.
View project
Visual Design
King Jewelers
A fresh, new website that reflects the King Jewelers brand and provides a premium online shopping experience
View project
UI Design
ConcepStore


In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Make Technology", "https://make.technology")

Found links: {'links': [{'type': 'home page', 'url': 'https://make.technology/'}, {'type': 'about page', 'url': 'https://make.technology/about/'}, {'type': 'contact page', 'url': 'https://make.technology/contact-us/'}, {'type': 'careers page', 'url': 'https://make.technology/careers/'}]}


"You are looking at a company called: Make Technology\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nMake Technology - Make Technology\nWebpage Contents:\nSkip to content\nContact Us\nAbout\nWebinars\nProjects\nArticles\nCareers\nSearch\nReady to make something exciting?\nLet's go\nCancel\nSearch\nMade to\nlast\nWe craft digital experiences that inspire, inform and entertain people.\nDiscover what we do\nUI Design\nConcepcion Industrial Corporation\nA website redesign for one of the leading air conditioning and refrigeration companies in the Philippines\nView project\nWorkshop\nGenerali\nGenerali, a Fortune 100 company, is a major player in the global insurance industry.\nView project\nVisual Design\nKing Jewelers\nA fresh, new website that reflects the King Jewelers brand and provides a premium online shopping experience\nView project\nUI Design\nConcepStore

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Make Technology", "https://make.technology")

Found links: {'links': [{'type': 'about page', 'url': 'https://make.technology/about/'}, {'type': 'careers page', 'url': 'https://make.technology/careers/'}, {'type': 'contact page', 'url': 'https://make.technology/contact-us/'}]}


# Make Technology Brochure

## About Us
At **Make Technology**, we are a spirited team of over 50 digital innovators dedicated to crafting digital experiences that inspire, inform, and entertain. As part of Ace Saatchi & Saatchi, our strength is further bolstered by a 100-strong communications team. Our motto is simple: make something amazing! We focus on creating solutions that are inclusive and accessible to everyone, regardless of technology's pace of change. 

We are proud to be ISO 9001:2015 certified, ensuring that our products and services meet international quality standards consistently.

## Our Services
We specialize in:

- **UI Design**
- **Visual Design**
- **Web Design**
- **Product Design**
- **Application Design**
- **Usability Testing**
- **Customer Experience Design**

Our approach is grounded in **Design Thinking**, allowing us to demystify the complexities of winning consumers’ hearts in the digital age.

## Featured Projects
We’ve collaborated with remarkable brands, creating bespoke solutions:

- **Concepcion Industrial Corporation**: Redesigning the website for a leader in air conditioning and refrigeration.
- **Generali**: A project for this Fortune 100 insurance giant.
- **King Jewelers**: Developing a premium online shopping experience through a refreshing website redesign.
- **Kingdom Golf**: Transitioning a luxury print magazine into a modern editorial lifestyle platform.

## Company Culture
At Make Technology, our culture revolves around collaboration, creativity, and innovation. We believe in empowering our team to think outside of the box while fostering a supportive environment to explore new ideas. We are passionate about what we do, and we take pride in our commitment to diversity and inclusion.

## Careers
Are you ready to make something amazing? Join our dynamic and innovative team! We are on the lookout for creative thinkers who are excited about digital innovation. Explore available job opportunities on our [Careers page](#).

## Get in Touch
Are you ready to take your brand to the next level? Connect with us, and let’s make something great together!

[Contact Us](#)

---

Make Technology: **Crafting Digital Innovations that Last**.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}



# Welcome to Hugging Face

## Who We Are
Hugging Face is a vibrant AI community dedicated to building the future through collaborative innovation in machine learning. Our platform serves as a hub where developers, researchers, and enthusiasts come together to create, share, and enhance models, datasets, and applications.

## Our Offering
- **Models**: Explore over **1,000,000 models**, ranging from advanced language models to image processing tools.
- **Datasets**: Access **250,000+ datasets** to fuel your machine learning projects.
- **Spaces**: Develop and host applications easily with our user-friendly interface and collaborative tools.
- **Enterprise Solutions**: We provide optimized compute resources and enterprise-grade support for over **50,000 organizations** including tech giants like Meta, Google, Microsoft, and AWS.

## Company Culture
At Hugging Face, our culture is deeply rooted in transparency, open-source collaboration, and community building. We prioritize a **supportive environment** where ideas can flow freely and everyone’s contribution is valued. Our team embraces diversity, encourages continuous learning, and believes in the power of collective growth in the AI landscape.

## Join Us
### Career Opportunities
We are always on the lookout for talented individuals who are passionate about AI and machine learning. Positions are available in various fields such as research, engineering, and business operations. Join us and be part of a community that is leading the charge in AI development.

**Explore Job Openings**: [Join Our Team](https://huggingface.co/jobs)

## Customer Testimonials
With a client base that spans across industries, Hugging Face is trusted by top enterprises and startups alike. Our solutions empower businesses to harness AI's potential and stay ahead in a rapidly evolving market.

## Connect With Us
Stay connected and find out more about what we are building:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

### Be Part of the Future of AI!
Join us in creating a collaborative platform that accelerates innovation in machine learning. Let's build the future, together.

---
*For more information, visit our website: [Hugging Face](https://huggingface.co)*



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>